<a href="https://colab.research.google.com/github/lunarcogito/synthpop_gan/blob/master/pretreatment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pypianoroll

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from pypianoroll import Multitrack, Track
import glob

def main():
  for filename in (glob.glob('drive/My Drive/Synth pop MIDI/*.mid')):
      mtrack = Multitrack(filename)
      l=len(mtrack.tracks)
      trackid=[x for x in range(l)]
      mtrack.to_pretty_midi()
      orchestra = group_by_program(mtrack)
      newmtrack = merge_instrument(mtrack,orchestra,trackid)
      #newmtrack = remove_short_tracks(track,5)
      #newmtrack.trim_trailing_silence()
      #newmtrack.clip(lower=60, upper=120)
      filename=filename[30:]
      print(filename,orchestra)
      newmtrack.write('drive/My Drive/Synth pop MIDI/result without drums/'+filename)

def remove_short_tracks(mtrack,amount):
  l=len(mtrack.tracks)
  i=j=l-1
  al=[]
  if l>amount:
      while j>=0:
          al.append((mtrack.tracks[j]).get_active_length())
          j-=1
      al.sort()
      a=[]
      a.append(al[l-1])
      while i>=0:
          actl=(mtrack.tracks[i]).get_active_length()
          apr=mtrack.tracks[i].get_active_pitch_range()
          print(apr)
          if actl<al[l-1]*0.4 or apr[1]<70:
              a.append(i)
          i-=1
      mtrack.remove_tracks(a)
  return mtrack
    

def group_by_program(mtrack):
  orchestra=[[]for _ in range(16)]
  trackcount=0
  for track in mtrack.tracks:
    if not track.is_drum:
      i=track.program // 8
      track.program=i*8
      orchestra[i].append(trackcount)
    trackcount+=1
  return orchestra

def merge_instrument(mtrack,orchestra,trackid):
  instname=['piano','chromatic percussion','organ','guitar','bass','strings','ensemble','brass','reed','pipe','synth lead','synth pad','synth effects','ethnic','percussive','sound effects']
  for p in range(0,16):
    if orchestra[p]!=[]:
      mtrack.merge_tracks(track_indices=orchestra[p], mode='sum', program=p, is_drum=False, name=instname[p], remove_merged=False)
  mtrack.remove_tracks(trackid)
  return mtrack

if __name__ == "__main__":
    main()